# calculating tpm based on formula provided [here](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/)

In [58]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [59]:
df = pd.read_csv('E:\\DATA\\microglia_sequencing\\GSE99622_hanamsagar2017_raw_reads.csv')
df2 = pd.read_excel('E:\\DATA\\microglia_sequencing\\Blbo_Mgla_RPKM_calculation_example.xlsx')

In [60]:
## subsetting the secondary dataframe to just grab the columns containing transcript length info for the merge

df2 = df2[['Transcript', 'Transcript length']]

In [61]:
df

,Gene,Transcript,Best,F_E18 1,M_E18 1,F_E18 3,M_E18 4,F_P14 1,F_P14 2,F_P14 3,...,F_P60_LPS 3,M_P60_LPS 3,M_P60_LPS 4,M_P60_LPS 5,M_P60_LPS 6,M_P60_Sal 1,M_P60_Sal 2,M_P60_Sal 3,M_P60_Sal 4,F_P60_Sal 6
0,Zfp85-rs1,NM_001001130,1,7,18,9,0,4,5,1,...,6,4,1,1,11,6,19,10,17,6
1,Scap,NM_001001144,1,133,181,132,53,98,152,141,...,64,26,35,39,59,121,207,128,110,8
2,Zfp458,NM_001001152,1,20,13,14,0,5,9,4,...,14,9,6,5,9,18,22,7,10,5
3,Fbxo41,NM_001001160,1,41,15,29,7,1,3,7,...,0,0,1,1,0,2,4,0,1,0
4,Taf9b,NM_001001176,1,38,32,37,15,30,43,64,...,8,18,30,18,31,65,41,52,55,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28639,4933401P06Rik,NR_045505,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
28640,4933405E24Rik,NR_045506,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28641,4933412O06Rik,NR_045507,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
28642,4933413L06Rik,NR_045508,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
df2

,Transcript,Transcript length
0,NM_001166372,4223
1,NM_001166375,3942
2,NM_175188,4391
3,NM_145486,3452
4,NM_177115,1754
...,...,...
28639,NM_001033634,10654
28640,NM_011777,2438
28641,NM_001045536,11150
28642,NM_001080755,7600


In [63]:
### merging the two dataframes by transcript column

df_merged = pd.merge(df, df2, on = 'Transcript')

In [64]:
df_merged.sort_values('Best', ascending = False, inplace = True)
df_merged.drop_duplicates(subset = 'Gene', keep = 'first', inplace = True)

df_merged.drop(columns = {'Best', 'Transcript'}, inplace = True)
df_merged.reset_index(drop=True, inplace=True)

In [65]:
df_merged

,Gene,F_E18 1,M_E18 1,F_E18 3,M_E18 4,F_P14 1,F_P14 2,F_P14 3,F_P14 4,F_P14 5,...,M_P60_LPS 3,M_P60_LPS 4,M_P60_LPS 5,M_P60_LPS 6,M_P60_Sal 1,M_P60_Sal 2,M_P60_Sal 3,M_P60_Sal 4,F_P60_Sal 6,Transcript length
0,Zfp85-rs1,7,18,9,0,4,5,1,5,4,...,4,1,1,11,6,19,10,17,6,2218
1,Snx7,44,25,38,74,14,23,29,29,49,...,6,11,8,10,27,18,19,30,18,2171
2,1700034O15Rik,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,741
3,Dnajc18,136,100,107,49,46,104,119,97,100,...,28,31,30,44,121,130,146,149,25,5167
4,Lce1i,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22864,Cdkn2aip,96,64,67,23,29,67,56,74,46,...,23,19,25,30,63,130,65,61,23,3494
22865,Phf20,187,116,126,7,87,126,106,124,82,...,66,42,38,104,171,209,115,143,28,5733
22866,Sox1,147,136,24,11,4,6,8,1,3,...,0,0,0,0,0,0,0,1,0,4037
22867,Mettl2,66,53,50,21,28,46,40,55,23,...,10,15,11,16,47,52,34,47,11,2619


In [66]:
### first we should drop any duplicate genes (only keep the uniques)

## i am doing that here by dropping rows that have a 'Best' value of 0
## then i get rid of the 'Best' and 'Transcript' columns, as they will not be helpful moving forward

# for i in range(len(df)):
#     if df_merged['Best'][i] == 0:
#         df_merged.drop(i, axis = 0, inplace = True)
        
# df_merged.drop(columns = {'Best', 'Transcript'}, inplace = True)


### Getting rid of this for now, trying the above method to try to conserve some more genes? idk we'll see

######## It doesn't seem to make any significant difference, let's keep the new solution as it is much faster!

In [67]:
## obtaining sample indices from columns in the dataframe

samples = df_merged.columns[1:-1]

In [68]:
### calculating RPK by dividing the raw read values by length of transcript

rpk = df_merged[samples].div(df_merged['Transcript length']/1000, axis=0)

In [69]:
### Count up all the RPK values in a sample and divide this number by 1,000,000. This is your “per million” scaling factor.

scaling_factor = np.zeros(shape = len(samples))
i = -1

for sample in samples:
    i = i + 1
    scaling_factor[i] = np.sum(rpk[sample])/1000000

In [70]:
### Divide the RPK values by the “per million” scaling factor. This gives you TPM.

tpm = rpk.copy()
i = -1

for sample in samples:
    i = i + 1
    tpm[sample] = rpk[sample]/scaling_factor[i]

In [71]:
tpm = tpm[tpm.all(axis = 1)]

In [72]:
tpm

,F_E18 1,M_E18 1,F_E18 3,M_E18 4,F_P14 1,F_P14 2,F_P14 3,F_P14 4,F_P14 5,F_P14 6,...,F_P60_LPS 3,M_P60_LPS 3,M_P60_LPS 4,M_P60_LPS 5,M_P60_LPS 6,M_P60_Sal 1,M_P60_Sal 2,M_P60_Sal 3,M_P60_Sal 4,F_P60_Sal 6
1,23.483806,15.983411,22.803118,37.541554,9.602956,10.898623,15.230369,11.664889,26.486588,18.840120,...,7.600523,8.695713,13.047346,10.118043,8.899140,13.455905,6.637133,10.861553,15.375287,19.361753
3,30.498331,26.862772,26.978374,10.444748,13.257331,20.706109,26.259155,16.393652,22.711787,22.617104,...,21.289897,17.050351,15.449433,15.942233,16.452129,25.337040,20.140616,35.068124,32.085558,11.298832
5,47.084528,29.497419,41.377352,10.545752,19.475028,27.628341,37.810631,26.919669,32.064832,44.348829,...,17.983052,31.596250,20.446622,14.748489,28.876276,38.153928,24.864105,35.070088,30.921609,33.812480
10,42.243002,14.627166,22.634425,9.724588,28.417162,36.625437,53.258051,41.287607,70.190483,64.905162,...,8.616163,4.480773,9.534586,3.128207,2.751358,23.112100,10.260051,33.934273,33.592074,5.986096
11,101.431354,102.939777,111.270714,35.485450,134.882960,107.876494,110.205551,120.504261,95.950258,79.921215,...,111.175163,69.813709,79.835047,107.661434,96.306551,79.255629,118.251927,84.121519,87.930438,26.262606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22854,43.912984,29.616549,46.053621,25.956278,13.278992,19.657362,16.703229,15.574024,12.333151,25.121732,...,12.261692,12.024453,12.301298,7.870081,19.073919,16.884002,17.845809,16.995618,17.008788,10.411909
22861,38.426685,46.915798,34.895842,35.823207,27.541568,26.899328,30.540453,20.606210,26.194599,16.769183,...,35.078092,18.059686,11.495855,24.515927,28.339335,28.290901,27.058123,33.243089,40.447398,40.211447
22864,31.836401,25.424146,24.981721,7.250121,12.359811,19.726750,18.274158,18.494869,15.449865,18.395654,...,28.335549,20.711840,14.002965,19.646421,16.588466,19.508623,29.784362,23.088122,19.425344,15.372232
22865,37.795083,28.084412,28.632486,1.344796,22.598210,22.609566,21.081241,18.887836,16.785013,11.211306,...,29.261725,36.222276,18.864994,18.199858,35.047680,32.271797,29.183152,24.895110,27.753380,11.405336


In [73]:
age = [0] * 60
tx = [0] * 60
sex = [0] * 60

i = -1
for sample in samples:
    i = i + 1
    if 'LPS' in sample:
        tx[i] = 'LPS'
    else:
        tx[i] = 'SAL'

i = -1
for sample in samples:
    i = i + 1
    if 'M' in sample:
        sex[i] = 'Male'
    else:
        sex[i] = 'Female'
        
i = -1
for sample in samples:
    i = i + 1
    if 'E18' in sample:
        age[i] = 'E18'
    if 'P4' in sample:
        age[i] = 'P4'
    if 'P14' in sample:
        age[i] = 'P14'
    if 'P60_LPS' in sample:
        age[i] = 'P60 + LPS'
    if 'P60_SAL' in sample:
        age[i] = 'P60 + SAL'
        
for i in range(len(age)):
    if age[i] == 0:
        age[i] = 'P60'
        
age = np.array(age).astype('str')

In [74]:
tpm.columns = [age, sex, tx, samples]

In [75]:
### reintroducing the gene column

tpm['gene'] = df_merged['Gene']

In [76]:
tpm.melt(id_vars = 'gene', var_name = ['age', 'sex', 'tx', 'sample'], value_name = 'expression')

,gene,age,sex,tx,sample,expression
0,Snx7,E18,Female,SAL,F_E18 1,23.483806
1,Dnajc18,E18,Female,SAL,F_E18 1,30.498331
2,Ipo11,E18,Female,SAL,F_E18 1,47.084528
3,Fam101b,E18,Female,SAL,F_E18 1,42.243002
4,Mgrn1,E18,Female,SAL,F_E18 1,101.431354
...,...,...,...,...,...,...
584635,Prune,P60,Female,SAL,F_P60_Sal 6,10.411909
584636,Wdr45,P60,Female,SAL,F_P60_Sal 6,40.211447
584637,Cdkn2aip,P60,Female,SAL,F_P60_Sal 6,15.372232
584638,Phf20,P60,Female,SAL,F_P60_Sal 6,11.405336


In [77]:
(tpm.melt(id_vars = 'gene', var_name = ['age', 'sex', 'tx', 'sample'], value_name = 'expression').
to_csv('C:\\Users\\Ben\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\shiny-app_200507\\GSE99622_hanamsagar2017_tpm_v2.csv'))

In [78]:
tpm.set_index('gene', inplace = True)

In [79]:
tpm.to_csv('C:\\Users\\Ben\\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi_w_rpkm\\GSE99622_hanamsagar2017_tpm_unmelted_v2.csv')